In [1]:
# myroot[5].attrib.get("sprecher") == 'Korinna Hennig'


import numpy as np
import pandas as pd 
from xml.dom import minidom
import xml.etree.ElementTree as ET
from nltk.tokenize import sent_tokenize
import nltk
import re
nltk.download('punkt')
import os
import PyPDF2


doc_list = []
interviewer = ""
fileNameList = []
questionList = []
interviewerinList = []


 
    

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Elias\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Diese Funktion findet heraus, um welchen Interviewer es sich handelt. Endsprechend wird der Name gespeichert und 
# zurückgegeben
# Zurückgegeben wird nur ein String, da pro Datei nur ein Name benötigt wird

def getInterviewer(text):
    
    if type(text) == str:
        
        text = text.split("\n")
    
    
    #text ist hier eine eingegebene Liste
    
    gefunden = False
    name = ""
    
    
    #Wir iterieren nur über die ersten 15. Zeilen. Das sollte reichen, um die Interviewerin zu bestimmen
    for line in text[:25]:
        if gefunden:
            
            return name
            break
        else:
         
            # Manche Elemente haben in der Liste ein Leerzeichen. Bsp ['Korinna Hennig ']. Dieses Leerzeichen führt zu
            # Fehlern beim Vergleich. Daher wird es mittels der strip()-Methode entfernt. upper() setzt alle Buchstaben groß-
            if line.upper().strip() == "KORINNA HENNIG" or line.upper().strip() == "CORINNA HENNIG"  :
                
                name = line.upper().strip()
                gefunden = True
                
               
                
            elif line.upper().strip() == "ANJA MARTINI":

                name = line.upper().strip()
                gefunden = True
                
            elif line.upper().strip() == "BEKE SCHULMANN":

                name = line.upper().strip()
                gefunden = True
               
               
                
            else:
                continue
                
        
    

# Diese Funktion findet heraus, um welchen Arzt es sich handelt. Endsprechend wird der Name gespeichert und 
# zurückgegeben
# Zurückgegeben wird nur ein String, da pro Datei nur ein Name benötigt wird
def getDoc(text):
    

    
    
    
    #text ist hier eine eingegebene Liste
    
    gefunden = False
    name = ""
    
    
    #Wir iterieren nur über die ersten 15. Zeilen. Das sollte reichen, um die Interviewerin zu bestimmen
    for line in text[:15]:
        if gefunden:
            
            return name
            break
        else:
         
            # Manche Elemente haben in der Liste ein Leerzeichen. Bsp ['Korinna Hennig ']. Dieses Leerzeichen führt zu
            # Fehlern beim Vergleich. Daher wird es mittels der strip()-Methode entfernt. upper() setzt alle Buchstaben groß-
            if line.upper().strip() == "CHRISTIAN DROSTEN" or line.upper().strip() == "CHRISTIAN DORSTEN":
                
                name = line.upper()
                gefunden = True
                
               
                
            elif line.upper().strip() == "SANDRA CIESEK":

                name = line.upper()
                gefunden = True
               
               
                
            else:
                name = "Weder D noch C"
                continue  
    
    
    
    
# Übergeben wird der Text der Sprecherin Korinna Hennig oder Anja Martini und nur ihre Fragen werden zurückgegeben
def filterNurFragen(text):
    
    # Der eingegebene Text wurde nochmal nach jedem einzelnen Satz getrennt
    tokened_text = sent_tokenize(text) 
    
    
   
    
    intern_list = []
   
    for sentence in tokened_text:

        #Where ^[A-Z] looks for a Capital at start
        # '[^?!.]*' means everything in between start and end is ok except things containing ? or ! or .
        # [?.!]$ must end with ? or ! or .
        
        #if re.match('^[A-Z][\w\s]+[?]$', sentence):
        
        if sentence.endswith("?"):
            intern_list.append(sentence)
            
    fragenAlsText =  " ".join(intern_list)
    
    
    return fragenAlsText
    
    
    
    
def get_interview_content(f, interviewer, arzt):
    
    
    if type(arzt) == str:
    
        #print("INTERVIEWERIN: ", interviewer)
        #print("BEFRAGTE PERSON: ", arzt)


        # creating an object 
        file = open(f, 'rb')
        print(file)

        # creating a pdf reader object
        fileReader = PyPDF2.PdfFileReader(file)




        # Erhalte die Anzahl an Seiten in der PDF
        NumPages = fileReader.getNumPages()

        interviewText = []   
        
       

        #Iteriere über alle Seiten in der PDF
        for i in range(0, NumPages):
            interViewFragen = False
            pageObj = fileReader.getPage(i)




            text = pageObj.extractText().split("\n")



            # Diese boolean-Abfrage wird NUR auf True gesetzt, wenn es sich um die Interviewerin hält. Wenn es D oder C sind, dann 
            #false. Und wenn es FALSE ist, wird der Satz nicht geprintet.

            

            for satz in text:
                if type(satz) == str:
                   

                    if satz.upper().startswith(interviewer):
                            interViewFragen = True
                    elif satz.upper().startswith(arzt):
                            interViewFragen = False

                    if interViewFragen is True:
                        interviewText.append(satz)
                else:
                    continue

            # Diese Funktion bereinigten den Interview-Text und gibt nur die Fragen zurück        
            cleaned_text = clean_text(interviewText,interviewer)
            #cleaned_text = "".join(interviewText)



        return cleaned_text
    else:
        return ""



# Hier übergeben wir den Text der Interviewerin und ihren Namen
# Der Name wird deshalb übergeben, um ihn aus der Liste zu entfernen, sodass nur die Fragen der Interviewerin in der Liste 
# enthalten sind


def clean_text(interview_Content, interviewer):
   


 # Dies wird als Index benutzt und gibt die Stelle an des Objektes in der Liste an, in welcher wir uns gerade befinden.
    index = 0

    for satz in interview_Content:
        if satz.upper() == interviewer:
            interview_Content.remove(satz)
        elif satz == "-":
            interview_Content.remove(satz)

        elif satz.startswith("-") or satz.endswith("-"):

            satz = satz.replace("-","")
            interview_Content[index] = satz

        index +=1


    bereinigterText = ("").join(interview_Content)
    return bereinigterText




## Prozesse
### Zunächst wird über alle Files iteriert und es wird für jede File Folgendes getan:


1.  Wir finden mit ``interviewerin = getInterviewer(firstPageofPDF)`` heraus, um welche Interviewerin (Anja Martini oder Korinna Hennig) es sich handelt
2. Punkt eins wiederholen wir erneut nur dieses Mal für den jeweiligen Arzt mithilfe der Zeile ``arzt = getDoc(firstPageofPDF)``
3. Jetzt filtern wir nur den Content der Interviewerin heraus.
4. Danach filtern wir diesen zuvor gefilterten Content nochmal nach Fragen
5. Abschließend werden die Daten aus den zuvor genannten Funktionen und die gefilterten Fragen in die drei Liste-Variablen ``doc_list = [] , fileNameList = [] , questionList = []`` abgespeichert, sodass daraus im Anschluss ein Dataframe erstellt werden kann.


# Umsetzung

In [3]:
#Begonnen wird mit #1 

directory = "Datensatz\PDF"


doc_list = []
interviewerin = ""
fileNameList = []
questionList = []



for filename in os.listdir(directory):
    if filename.endswith(".pdf"): 
        
        print("[DATEINAME:] " ,filename)
        #print(" ------------------------- \n")
        
        # file ist die jeweilige Datei
        file = os.path.join(directory, filename)
        
       
        
        fileReader = PyPDF2.PdfFileReader(file)
        
        # Wir nehmen die ersten Seite, um Interviewerin und Arzt / Ärztin zu bestimmen
        
        pageObj = fileReader.getPage(0)
        firstPageofPDF = pageObj.extractText().split("\n")
        
        interviewerin = getInterviewer(firstPageofPDF)
        
        
        arzt = getDoc(firstPageofPDF)
        
       
        # Hier wird NUR der Inhalt der Interviewerin erstmal gefiltert
        only_interview_content = get_interview_content(file, interviewerin, arzt)
        
        
        # Der zuvor gefilterte Inhalt wird jetzt nur nochmal auf Fragen reduziert
        only_questions = filterNurFragen(only_interview_content)
        
        
        
        # Hier wird nochmal alles in die jeweiligen Listen abgespeichert
        
        # SPEICHER DIE ÄRZTE DER JEWEILIGEN FILE IN EINE LISTE 
        doc_list.append(arzt)
        
        
        
        interviewerinList.append(interviewerin)
        
        
        # FÜGE FRAGEN DER FRAGENLISTE HINZU
        #print("ONLY: ", only_questions)
        questionList.append(only_questions)
        
        # SPEICHER DEN TITEL DER FILE IN EINE LISTE
        fileNameList.append(filename)
        
        
        #print("--- DOKUMENT ENDE ----  \n\n\n\n")
        
        #print(ergebnis)
        
       
        
        
        #print((firstPageofPDF[:15]))
        
        
        
        
        # Nr. 2
        
            

# creating a pdf reader object
   


        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
    else:
        print("Keine PDF-Datei!")
   

[DATEINAME:]  10_Großveranstaltungen absagen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\10_Großveranstaltungen absagen.pdf'>
[DATEINAME:]  11_Wir müssen jetzt gezielt handeln.pdf
<_io.BufferedReader name='Datensatz\\PDF\\11_Wir müssen jetzt gezielt handeln.pdf'>
[DATEINAME:]  12_Schulen schließen und Gemeinden unterstützen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\12_Schulen schließen und Gemeinden unterstützen.pdf'>
[DATEINAME:]  13_Natürlich kann man noch einkaufen gehen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\13_Natürlich kann man noch einkaufen gehen.pdf'>


[DATEINAME:]  14_Vorsicht vor Vereinfachungen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\14_Vorsicht vor Vereinfachungen.pdf'>
[DATEINAME:]  15_Infizierte werden offenbar immun.pdf
<_io.BufferedReader name='Datensatz\\PDF\\15_Infizierte werden offenbar immun.pdf'>
[DATEINAME:]  16_Wir brauchen Abkürzungen bei der Impfstoffzulassung.pdf
<_io.BufferedReader name='Datensatz\\PDF\\16_Wir brauchen Abkürzungen bei der Impfstoffzulassung.pdf'>
[DATEINAME:]  17_Malaria-Medikament vorerst kein Hoffnungsträger.pdf
<_io.BufferedReader name='Datensatz\\PDF\\17_Malaria-Medikament vorerst kein Hoffnungsträger.pdf'>
[DATEINAME:]  18_Wirksamkeit von Ausgangssperren ist unklar.pdf
<_io.BufferedReader name='Datensatz\\PDF\\18_Wirksamkeit von Ausgangssperren ist unklar.pdf'>
[DATEINAME:]  19_Masken können andere schützen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\19_Masken können andere schützen.pdf'>
[DATEINAME:]  1_Wir können die Ausbreitung verlangsamen.pdf
<_io.BufferedReader name='Datensatz\\P

[DATEINAME:]  61_Winter is coming - Skript herunterladen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\61_Winter is coming - Skript herunterladen.pdf'>
[DATEINAME:]  61_Winter is coming.pdf
<_io.BufferedReader name='Datensatz\\PDF\\61_Winter is coming.pdf'>
[DATEINAME:]  62_Der Überlastschalter - Skript herunterladen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\62_Der Überlastschalter - Skript herunterladen.pdf'>
[DATEINAME:]  62_Der Überlastschalter.pdf
<_io.BufferedReader name='Datensatz\\PDF\\62_Der Überlastschalter.pdf'>
[DATEINAME:]  63_Risikogruppen kann man nicht wegsperren - Skript herunterladen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\63_Risikogruppen kann man nicht wegsperren - Skript herunterladen.pdf'>
[DATEINAME:]  63_Risikogruppen kann man nicht wegsperren.pdf
<_io.BufferedReader name='Datensatz\\PDF\\63_Risikogruppen kann man nicht wegsperren.pdf'>
[DATEINAME:]  64_Keine Angst vor Mutationen - Skript herunterladen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\64_Ke

[DATEINAME:]  87_Die zweite Spritze macht den Unterschied.pdf
<_io.BufferedReader name='Datensatz\\PDF\\87_Die zweite Spritze macht den Unterschied.pdf'>
[DATEINAME:]  88_Impfmission possible - Skript herunterladen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\88_Impfmission possible - Skript herunterladen.pdf'>
[DATEINAME:]  88_Impfmission possible.pdf
<_io.BufferedReader name='Datensatz\\PDF\\88_Impfmission possible.pdf'>
[DATEINAME:]  89_Gemeinsam gegen die Pandemie - Skript herunterladen.pdf
<_io.BufferedReader name='Datensatz\\PDF\\89_Gemeinsam gegen die Pandemie - Skript herunterladen.pdf'>
[DATEINAME:]  89_Gemeinsam gegen die Pandemie.pdf
<_io.BufferedReader name='Datensatz\\PDF\\89_Gemeinsam gegen die Pandemie.pdf'>
[DATEINAME:]  8_Viren mutieren immer.pdf
<_io.BufferedReader name='Datensatz\\PDF\\8_Viren mutieren immer.pdf'>
[DATEINAME:]  90_Das Virus auf der Standspur.pdf
<_io.BufferedReader name='Datensatz\\PDF\\90_Das Virus auf der Standspur.pdf'>
[DATEINAME:]  91_Die Pande

# DATENBLATT ERSTELLEN 

In [6]:
import pandas as pd

pd.set_option('display.max_rows', None)
  


# intialise data of lists.
data = {'Filename': fileNameList,
        'Questions': questionList,
        'Interviewerin': interviewerinList,
        'Arzt' :  doc_list      }
  
# Create DataFrame

df = pd.DataFrame(data)



In [10]:
questionList[2]

'Herr Drosten, ist das Einreiseverbot die richtige Entscheidung von US Präsident Trumps, um sein Land zu schützen? Ist es der richtige Weg oder ist es ein bisschen übertrieben? Würden Sie sagen: Schulschließungen - ja? Aber wie lösen wir jetzt das Problem? In der aktuellen Situation, würden Sie sagen: Bleibt besser zu Hause? Oder: Macht nichts, geht raus? Wenn wir aber aufs Land gehen und auf die kleinen Kliniken schauen: Wie sind die ausgestattet?'

In [8]:
df

,Filename,Questions,Interviewerin,Arzt
0,10_Großveranstaltungen absagen.pdf,Warum müssen es 1000 Menschen sein? Warum könn...,ANJA MARTINI,CHRISTIAN DROSTEN
1,11_Wir müssen jetzt gezielt handeln.pdf,Was wäre es? Die Frage war: Ist es noch sinnvo...,ANJA MARTINI,CHRISTIAN DROSTEN
2,12_Schulen schließen und Gemeinden unterstütze...,"Herr Drosten, ist das Einreiseverbot die richt...",ANJA MARTINI,CHRISTIAN DROSTEN
3,13_Natürlich kann man noch einkaufen gehen.pdf,Die machen sich jetzt Gedanken: Wie schütze ic...,ANJA MARTINI,CHRISTIAN DROSTEN
4,14_Vorsicht vor Vereinfachungen.pdf,Hat es geklappt? Ist da was dran? Aber was bed...,ANJA MARTINI,CHRISTIAN DROSTEN
5,15_Infizierte werden offenbar immun.pdf,Wie gut funktioniert das bei Ihrem Team in der...,KORINNA HENNIG,CHRISTIAN DROSTEN
6,16_Wir brauchen Abkürzungen bei der Impfstoffz...,Eine der meistgestellten Hörerfragen ist mittl...,KORINNA HENNIG,CHRISTIAN DROSTEN
7,17_Malaria-Medikament vorerst kein Hoffnungstr...,Bringt so was was? Das haben wir aus unseren M...,ANJA MARTINI,CHRISTIAN DROSTEN
8,18_Wirksamkeit von Ausgangssperren ist unklar.pdf,"Lassen Sie uns klarstellen, warum das keine Em...",KORINNA HENNIG,CHRISTIAN DROSTEN
9,19_Masken können andere schützen.pdf,Wie haben Sie das Wochenende erlebt? Wie haben...,KORINNA HENNIG,CHRISTIAN DROSTEN


# Datenblatt für Kurs-Umfrage

In [11]:
# Nach Ausführen der Zeile wird eine neue Datei erstellt im selben Ordnerverzeichnis, die die Spalte "Ärzte" nicht enthölt

In [12]:
df.to_csv('file_1.csv',encoding='utf-8-sig')

In [13]:
datei = "Datensatz/PDF/29_Tests müssen gezielter eingesetzt werden.pdf"
fileReader1 = PyPDF2.PdfFileReader(datei)
        
        # Wir nehmen die ersten Seite, um Interviewerin und Arzt / Ärztin zu bestimmen
        
pageObj1 = fileReader1.getPage(0)
firstPageofPDF1 = pageObj1.extractText()
print(firstPageofPDF1)


1/6
12KORINNA HENNIG
WISSENSCHAFTSREDAKTEURIN, NDR INFO
CHRISTIAN DROSTEN
VIROLOGE, CHARITE BERLIN
CORONAVIRUS-UPDATE 
FOLGE 29
Korinna Hennig 
Für viele beginnt der Tag ja mit der Zahl der gemeldeten 
Neuinfektionen. Und wir wissen, dass das eine Informati
-on unter Vorbehalt ist, weil die Zahl der nicht erkannten, 


-ten Zahlen, je nach Quelle. Das Robert Koch-Institut 




-
auf regionaler Ebene recherchieren, allerdings auch 


Christian Drosten


bedienen sich unterschiedlicher Quellen, die ich so gar 




ist nicht nur in Deutschland so übrigens! Ich habe da in 



-



-

Zahlen dabei, die kommen aus Zeitungsmeldungen. Da 






-
wäre das das RKI, und korrigieren sich sicherlich auch 


-
DAS RKI IST DIE ENTSCHEIDENDE 
 QUELLE FÜR DIE ZAHLEN 
Das ist alles auch nicht falsch. Es ist nicht so wichtig, ob 

-



-
-
in Deutschland die Zahlen des Robert Koch-Instituts, 
und es ist vollkommen egal, ob die in der elektronischen 

-



-
Korinna Hennig

-
-lich schon Erkenntnisse dar

'KORINNA HENNIG'

Datensatz/PDF/29_Tests müssen gezielter eingesetzt werden.pdf
<_io.BufferedReader name='Datensatz/PDF/29_Tests müssen gezielter eingesetzt werden.pdf'>


NameError: name 'test' is not defined

'Korinna Hennig Für viele beginnt der Tag ja mit der Zahl der gemeldeten Neuinfektionen. Und wir wissen, dass das eine Information unter Vorbehalt ist, weil die Zahl der nicht erkannten, ten Zahlen, je nach Quelle. Das Robert KochInstitut auf regionaler Ebene recherchieren, allerdings auch lich schon Erkenntnisse darüber, wo gegenwärtig die sie eine Infektion gar nicht bemerken. Es wird immer Charité auch beteiligt ist.Korinna Hennig über Kläranlagen so ein Infektionsgeschehen nachverKorinna Hennig viele nach. Es gibt im Zusammenhang mit Bakterien -'

''

'KORINNA HENNIG'